In [ ]:
%matplotlib inline
#%pylab
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mpc
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

import scipy
from scipy import interpolate
import pandas as pd
import itertools
from root_numpy import root2array, root2rec, tree2rec, array2root
from ROOT import TChain

In [ ]:
def open_df(fname,treename="flashtrigger/opflash_opflash_tree",start=None,stop=None):
    return pd.DataFrame( root2array(fname, treename,start=start,stop=stop) )

In [ ]:
#
# Data loading
#
df_bnb = open_df('flash_trigger_ana/bnb.root').query('0<dt and dt<1.6')



In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

interval = 10.
eff1_v=[]
eff1_err_v=[]
eff2_v=[]
eff2_err_v=[]
x_err_v=[]
for x in xrange(25):
    xmin=x*interval
    xmax=(x+1)*interval
    x_err_v.append(interval/2.)
    cut_str = '%g < nu_mc_x and nu_mc_x < %g' % (xmin,xmax)
    
    total = float(len(df_bnb.query(cut_str).groupby(['run','subrun','event'])))    
    sel = float(len(df_bnb.query('%s and pe_total>50' % cut_str).groupby(['run','subrun','event'])))
    eff1_v.append(sel/total)
    eff1_err_v.append(np.sqrt(sel*(1-sel/total))/total)
    
    sel = float(len(df_bnb.query('%s and pe_total>2' % cut_str).groupby(['run','subrun','event'])))
    eff2_v.append(sel/total)
    eff2_err_v.append(np.sqrt(sel*(1-sel/total))/total)
   
plt.errorbar(np.arange(5,255,10),eff2_v,yerr=eff2_err_v,xerr=x_err_v,
             label='Cut > 2 P.E.',color='red',marker='o')
plt.errorbar(np.arange(5,255,10),eff1_v,yerr=eff1_err_v,xerr=x_err_v,
             label='Cut > 50 P.E.',color='blue',marker='o')

ax.legend(loc=3,prop={'size':20})
plt.ylim(0.5,1.05)
plt.xlim(0,250)
plt.xlabel('X [cm]',fontsize=20)
plt.ylabel('Efficiency',fontsize=20)
plt.tick_params(labelsize=20)
plt.grid()
plt.title('OpFlash Efficiency Distribution (MCC 6.1 BNB)',fontsize=20)
plt.show()

In [ ]:
#
# X-Y distribution of efficiency
#
df_bnb_low  = df_bnb.query('pe_total>2')
df_bnb_high = df_bnb.query('pe_total>50')

nbinsx = 25
nbinsz = 50

h2d_low,xbins_low,ybins_low = np.histogram2d(df_bnb_low['nu_mc_x'].ravel(),df_bnb_low['nu_mc_z'].ravel(),
                                             bins=(nbinsx,nbinsz),range=((0,250),(0,1030)))

h2d_high,xbins_high,ybins_high = np.histogram2d(df_bnb_high['nu_mc_x'].ravel(),df_bnb_high['nu_mc_z'].ravel(),
                                                 bins=(nbinsx,nbinsz),range=((0,250),(0,1030)))

efficiency = h2d_high / h2d_low

fig, ax = plt.subplots(figsize=(12, 8))

# Let mpl figure out histograming (alternative: numpy.histogram2d)
#data, xbins, ybins, im = ax.hist2d(x,y,bins=(100,100),range=(range1,range2),norm = mpc.LogNorm())

#Configure color bar
#im.cmap.set_under('none')
#cbar = fig.colorbar(im)
#cbar.set_label("Selected Shower Counts")

# Manupulate axis through pyplot as I haven't figured out how-to from Axes to set fontsize
plt.xlabel('Z [cm]',fontsize=20)
plt.ylabel('X [cm]',fontsize=20)
plt.title('BNB (MCC 6.1) 50 > P.E. Cut Efficiency Z vs. X',fontsize=20)
ax.tick_params(labelsize=20)
plt.imshow(efficiency,extent=[0,1030,250,0], interpolation='nearest',aspect=2)   
cbar=plt.colorbar(shrink=0.6,ticks=[0,0.5,1])
cbar.ax.tick_params(labelsize=15)
cbar.ax.set_ylabel('Efficiency', fontsize=20)
# Visualize
plt.tight_layout()
ax.tick_params(labelsize=15)
plt.show()
